<a href="https://colab.research.google.com/github/LuizHenrique21/ia-grafos-luiz-henrique/blob/main/ia_grafos_buscas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo:** Luiz Henrique Bezerra Almeida\
**Matricula:** 2401905\
**Turma:** ENGCO221N01

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?

💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [ ]:
import heapq

def dijkstra(grafo, inicio):
    dist = {no: float('inf') for no in grafo}
    parent = {no: None for no in grafo}

    dist[inicio] = 0
    heap = [(0, inicio)]

    while heap:
        dist_atual, no_atual = heapq.heappop(heap)

        if dist_atual > dist[no_atual]:
            continue

        for vizinho, peso in grafo[no_atual]:
            nova_dist = dist_atual + peso
            if nova_dist < dist[vizinho]:
                dist[vizinho] = nova_dist
                parent[vizinho] = no_atual
                heapq.heappush(heap, (nova_dist, vizinho))

    return dist, parent


In [ ]:
def reconstruct_path(parent, target):
    caminho = []
    atual = target
    while atual is not None:
        caminho.append(atual)
        atual = parent[atual]
    caminho.reverse()
    return caminho


In [ ]:
grafo = {
    'Hospital': [('A', 4), ('B', 2)],
    'A': [('Hospital', 4), ('C', 3)],
    'B': [('Hospital', 2), ('C', 1), ('D', 7)],
    'C': [('A', 3), ('B', 1), ('D', 2), ('Local', 5)],
    'D': [('B', 7), ('C', 2), ('Local', 1)],
    'Local': [('C', 5), ('D', 1)]
}

dist, parent = dijkstra(grafo, 'Hospital')
caminho = reconstruct_path(parent, 'Local')

print("Custo mínimo:", dist['Local'])
print("Caminho mais rápido:", caminho)


Custo mínimo: 6
Caminho mais rápido: ['Hospital', 'B', 'C', 'D', 'Local']


**Por que Dijkstra exige arestas não negativas?**\
Porque o algoritmo assume que, ao visitar um nó com a menor distância acumulada, essa distância é definitiva. Se houvesse arestas negativas, poderia existir um caminho mais curto que aparece depois, quebrando essa propriedade e causando erros na atualização das distâncias.


**Qual a complexidade do algoritmo com lista de adjacência e heapq?**\
Cada vértice é inserido no heap pelo menos uma vez e removido uma vez: O(V log V).

Cada aresta é examinada no máximo uma vez, com operações de atualização no heap: O(E log V).

Portanto, a complexidade total é O((V + E) log V), onde V é o número de vértices e E o número de arestas.

**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?

2. Por que a heurística Manhattan é admissível nesse caso?

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [ ]:
import random

def criar_grid(tamanho=20, porcentagem_obstaculos=0.15):
    grid = [[0 for _ in range(tamanho)] for _ in range(tamanho)]
    total_celulas = tamanho * tamanho
    num_obstaculos = int(total_celulas * porcentagem_obstaculos)

    obstáculos_pos = set()
    while len(obstáculos_pos) < num_obstaculos:
        x = random.randint(0, tamanho - 1)
        y = random.randint(0, tamanho - 1)
        obstáculos_pos.add((x, y))

    for (x, y) in obstáculos_pos:
        grid[x][y] = 1

    return grid


In [ ]:
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])


In [ ]:
import heapq

def a_star(grid, start, goal, h):
    tamanho = len(grid)
    open_set = []
    heapq.heappush(open_set, (0 + h(start, goal), 0, start))

    came_from = {}
    g_score = {start: 0}

    movimentos = [(0,1),(1,0),(-1,0),(0,-1)]

    while open_set:
        f_atual, g_atual, atual = heapq.heappop(open_set)

        if atual == goal:
            caminho = []
            while atual in came_from:
                caminho.append(atual)
                atual = came_from[atual]
            caminho.append(start)
            caminho.reverse()
            return caminho

        for dx, dy in movimentos:
            vizinho = (atual[0] + dx, atual[1] + dy)

            if 0 <= vizinho[0] < tamanho and 0 <= vizinho[1] < tamanho:
                if grid[vizinho[0]][vizinho[1]] == 1:
                    continue

                tentative_g = g_atual + 1

                if tentative_g < g_score.get(vizinho, float('inf')):
                    came_from[vizinho] = atual
                    g_score[vizinho] = tentative_g
                    f_score = tentative_g + h(vizinho, goal)
                    heapq.heappush(open_set, (f_score, tentative_g, vizinho))

    return None


In [ ]:
def imprimir_grid(grid, caminho=None):
    for i in range(len(grid)):
        linha = ""
        for j in range(len(grid[0])):
            if caminho and (i, j) in caminho:
                linha += "🟢"
            elif grid[i][j] == 1:
                linha += "⬛"
            else:
                linha += "⬜"
        print(linha)

grid = criar_grid()
start = (0, 0)
goal = (19, 19)

caminho = a_star(grid, start, goal, heuristic)

if caminho:
    print(f"Caminho encontrado com {len(caminho)} passos:")
    imprimir_grid(grid, caminho)
else:
    print("Nenhum caminho encontrado.")


Caminho encontrado com 39 passos:
🟢🟢🟢🟢🟢🟢🟢🟢🟢🟢⬛⬜⬛⬜⬜⬜⬛⬜⬜⬜
⬜⬜⬛⬜⬛⬜⬜⬜⬜🟢🟢🟢⬜⬜⬜⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜🟢⬜⬜⬜⬜⬜⬛⬜⬛
⬛⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜🟢⬜⬛⬛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜🟢⬛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬜⬜⬛⬜⬜⬜🟢🟢🟢⬜⬜⬜⬜⬜⬜
⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🟢⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜⬛⬜🟢⬛⬛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜🟢🟢🟢⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜🟢⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜🟢⬛⬜⬜⬜
⬜⬛⬜⬜⬜⬛⬜⬜⬜⬜⬜⬛⬜⬜⬜🟢🟢⬛⬛⬜
⬜⬜⬛⬛⬜⬛⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜🟢⬛⬜⬜
⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🟢⬛⬜⬛
⬜⬜⬜⬜⬛⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🟢🟢⬜⬜
⬜⬜⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬛⬜⬜⬜⬛🟢⬛⬜
⬜⬛⬛⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬛⬜⬜🟢🟢⬛
⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛🟢🟢
⬜⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬜⬜⬜⬜⬜🟢
⬜⬛⬜⬜⬜⬜⬜⬜⬜⬜⬛⬜⬛⬜⬜⬜⬜⬜⬜🟢


**A * vs Dijkstra: qual expande menos nós?**\
A* expande menos nós porque usa a heurística para priorizar os nós mais promissores (mais perto do objetivo). Dijkstra é uma busca cega, explorando uniformemente, enquanto o A* é uma busca informada.

**Por que a heurística Manhattan é admissível nesse caso?**

Porque a distância Manhattan nunca superestima o custo real para chegar ao destino num grid onde só se pode mover para cima, baixo, esquerda ou direita, sem atravessar obstáculos. Ou seja, é sempre menor ou igual ao custo real, garantindo que A* encontre o caminho ótimo.

**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?

In [ ]:
class Node:
    def __init__(self, valor):
        self.valor = valor
        self.esq = None
        self.dir = None

def inserir(root, valor):
    if root is None:
        return Node(valor)
    if valor < root.valor:
        root.esq = inserir(root.esq, valor)
    else:
        root.dir = inserir(root.dir, valor)
    return root

valores = [50, 30, 70, 20, 40, 60, 80, 35, 45]
root = None
for v in valores:
    root = inserir(root, v)

def in_order(root):
    if root is None:
        return []
    return in_order(root.esq) + [root.valor] + in_order(root.dir)

def pre_order(root):
    if root is None:
        return []
    return [root.valor] + pre_order(root.esq) + pre_order(root.dir)

def post_order(root):
    if root is None:
        return []
    return post_order(root.esq) + post_order(root.dir) + [root.valor]

print("In-order:", in_order(root))
print("Pre-order:", pre_order(root))
print("Post-order:", post_order(root))


In-order: [20, 30, 35, 40, 45, 50, 60, 70, 80]
Pre-order: [50, 30, 20, 40, 35, 45, 70, 60, 80]
Post-order: [20, 35, 45, 40, 30, 60, 80, 70, 50]


**Em que situação cada tipo de percurso é mais indicado?**

In-order: ideal para ordenar elementos em uma BST, pois visita os nós em ordem crescente.

Pre-order: útil para clonar ou serializar a árvore, pois visita primeiro a raiz e depois os filhos, preservando a estrutura.

Post-order: indicado para operações que dependem da visita dos filhos antes do pai, como liberação de memória (em linguagens que requerem isso) ou para calcular totais agregados de subárvores.

**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

2. Diferencie corretude e otimalidade nos algoritmos estudados.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

**Quando não é vantajoso usar A, mesmo tendo uma heurística?**

No contexto de jogos de estratégia com mapas muito dinâmicos e ambientes que mudam rápido, usar A* pode não ser vantajoso se a heurística não acompanhar as alterações do terreno. Por exemplo, se a heurística estima a distância como se não houvesse obstáculos recentes (como barreiras móveis), o A* pode fazer muitos cálculos desnecessários e acabar lento. Também em mapas densos, onde há muitos caminhos similares, a heurística pode não diferenciar bem os nós, fazendo com que o A* explore quase tanto quanto o Dijkstra (busca uniforme). Além disso, em sistemas de busca em que calcular a heurística exige análise complexa (ex: árvores sintáticas gigantes em compiladores), o custo do cálculo pode superar o benefício.

**Diferencie corretude e otimalidade nos algoritmos estudados.**

Em sistemas de navegação por mapas, corretude garante que o algoritmo sempre vai encontrar um caminho válido entre dois pontos se ele existir, como um GPS mostrando uma rota possível. Já otimalidade significa que o caminho encontrado é o mais rápido ou curto possível, como o Google Maps indicando a rota de menor tempo. Por exemplo, o algoritmo A* com heurística admissível é correto e ótimo porque não só encontra um caminho, mas garante o melhor caminho. Em jogos, uma busca pode ser correta (não travar) mas não ótima se usar heurísticas ruins que resultam em rotas mais longas. Na análise de árvores sintáticas, a corretude garante que a árvore está bem formada, mas a otimalidade pode não ser relevante.

**Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.**

Em-ordem: Em sistemas de mapas que organizam pontos de interesse por ordem de distância ou preço, percorrer uma árvore binária em ordem permite listar esses pontos em ordem crescente, como exibir hotéis do mais barato ao mais caro.

Pré-ordem: Nos jogos, ao salvar o estado do mapa ou da árvore de decisões, o percurso pré-ordem é usado para serializar o estado, salvando primeiro a posição atual do jogador e depois suas possíveis ações, facilitando o carregamento posterior.

Pós-ordem: Em sistemas de compiladores, ao interpretar expressões matemáticas em árvores sintáticas, o percurso pós-ordem avalia primeiro os operandos (nós filhos) antes de aplicar a operação do nó pai, garantindo que a expressão seja calculada corretamente.

**Como heurísticas inconsistentes podem afetar o resultado do A?**

Em jogos com mapas grandes, uma heurística inconsistente pode fazer o A* “voltar atrás” várias vezes ao encontrar novos obstáculos ou terrenos difíceis, fazendo o personagem recalcular o caminho repetidamente. Isso torna a busca menos eficiente, pois o algoritmo revisita nós várias vezes, atrasando a tomada de decisão. Em sistemas de busca de árvores sintáticas, heurísticas inconsistentes podem levar a reanálises de partes já processadas, aumentando o tempo de compilação sem melhorar o resultado final. Apesar disso, o A* continua encontrando o caminho ótimo, só que com custo computacional maior.

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.